### Esse script serve para extrair as informações presentes no site SteamDB de maneira automatizada e exportá-los a uma planilha Excel.
### Lembro que para executar o script, é necessário instalar as bibliotecas necessárias.
### Versão do Python: **3.12.4**.

In [ ]:
# Para instalar as bibliotecas é só tirar o cometário abaixo
# !pip install selenium seleniumbase pandas numpy

### Importação das Bibliotecas

In [4]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

### WebScrapping do site da SteamDB 

In [ ]:
# Criando um webdriver
driver = Driver(uc=True)

# Url pra acessar o site do steamDB
url = 'https://steamdb.info/sales/?min_reviews=0'

# Passando pelo captcha
driver.uc_open_with_reconnect(url, 4)
driver.uc_gui_click_captcha()

# Timer pra carregar a página
time.sleep(10)

# Clicar no botão para mostrar todos os itens da página
driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[1]/div[1]/select/option[8]').click()

# Total de itens na página
itens = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[1]/div/div/div/div[1]/h2/span').text.replace('.', '')
print('Total de itens:', itens)

# Timer pra carregar todos os itens da página
time.sleep(20)

# Dicionário de listas para adicionar os dados
dados = {
    'Nome': [],
    'Data de Lançamento': [],
    'Avaliação (%)': [],
    'Preço (R$)': [], 
    'Qtd de Desconto (%)': [],
    'Inicio do Desconto': [],
    'Fim do Desconto': []
}

# Loop para adicionar os dados de cada item
for item in range(1, int(itens) + 1):
    # Atribuindo os dados do item a cada variável relacionada
    nome = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[3]/a').text
    lancamento = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[7]').get_attribute('data-sort')                                   
    avaliacao = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[6]').get_attribute('data-sort')
    preco = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[5]').get_attribute('data-sort')
    desconto = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[4]').get_attribute('data-sort')
    inicio_desconto = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[9]').get_attribute('data-sort')
    fim_desconto = driver.find_element(By.XPATH,f'/html/body/div[4]/div[1]/div[2]/div[1]/div[2]/div[4]/div/div[2]/table/tbody/tr[{item}]/td[8]').get_attribute('data-sort')

    # Adição do valor das variáveis relacionadas do item
    dados['Nome'].append(nome)
    dados['Data de Lançamento'].append(lancamento)
    dados['Avaliação (%)'].append(avaliacao)
    dados['Preço (R$)'].append(preco)
    dados['Qtd de Desconto (%)'].append(desconto)
    dados['Inicio do Desconto'].append(inicio_desconto)
    dados['Fim do Desconto'].append(fim_desconto)

    print(f'Dados do item {item}/{itens} adicionados com sucesso')

# Fechar o navegador depois de realizar a extração
driver.close()

### Tratamento dos Dados

In [16]:
# Criando um Dataframe com os dados extraído do SteamDB
df_dados = pd.DataFrame(dados)
df_dados

,Nome,Data de Lançamento,Avaliação (%),Preço (R$),Qtd de Desconto (%),Inicio do Desconto,Fim do Desconto
0,Age of Wonders 4,1682985600,79.52,12999,35,1731002536,1731607200
1,ICARUS,1638489600,70.52,5099,50,1731002514,1731607200
2,Age of Water,1714003200,62.62,7450,50,1731002576,1731607200
3,Baldur's Gate 3,1691020800,95.98,15999,20,1730744570,1731348000
4,Minishoot' Adventures,1712016000,93.96,2999,25,1731002534,1731607200
...,...,...,...,...,...,...,...
4811,Spirit Mancer,1732233600,-1,3149,10,1727946002,1732298400
4812,Bottle,1664323200,-1,782,10,1730744667,1731348000
4813,Grid Tales,1639440000,-1,1673,10,1730744634,1731348000
4814,My Wet Leto Comic-A Day of Green,1730332800,-1,2699,10,1730387402,1731607200


In [17]:
# Transformando a coluna de Avaliação para float e os valores -1 em NaN
df_dados['Avaliação (%)'] = df_dados['Avaliação (%)'].astype(float).replace(-1, np.nan)

# Transformando a coluna de preço em inteiro e dividindo por 100 para retratar os centavos
df_dados['Preço (R$)'] = df_dados['Preço (R$)'].astype(int) / 100

# Transformando a coluna de Desconto em inteiro
df_dados['Qtd de Desconto (%)'] = df_dados['Qtd de Desconto (%)'].astype(int)

# Transformando a coluna de Data de Lançamento de Timestamp para Datetime
df_dados['Data de Lançamento'] = pd.to_datetime(df_dados['Data de Lançamento'], unit='s')

# Transformando o Inicio de Desconto de Timestamp para Datetime
df_dados['Inicio do Desconto'] = pd.to_datetime(df_dados['Inicio do Desconto'], unit = 's')

# Transformando o Fim de Desconto de Timestamp para Datetime
df_dados['Fim do Desconto'] = pd.to_datetime(df_dados['Fim do Desconto'], unit = 's')

In [18]:
# Dataset final com tratamento dos dados extraídos
df_dados

,Nome,Data de Lançamento,Avaliação (%),Preço (R$),Qtd de Desconto (%),Inicio do Desconto,Fim do Desconto
0,Age of Wonders 4,2023-05-02,79.52,129.99,35,2024-11-07 18:02:16,2024-11-14 18:00:00
1,ICARUS,2021-12-03,70.52,50.99,50,2024-11-07 18:01:54,2024-11-14 18:00:00
2,Age of Water,2024-04-25,62.62,74.50,50,2024-11-07 18:02:56,2024-11-14 18:00:00
3,Baldur's Gate 3,2023-08-03,95.98,159.99,20,2024-11-04 18:22:50,2024-11-11 18:00:00
4,Minishoot' Adventures,2024-04-02,93.96,29.99,25,2024-11-07 18:02:14,2024-11-14 18:00:00
...,...,...,...,...,...,...,...
4811,Spirit Mancer,2024-11-22,NaN,31.49,10,2024-10-03 09:00:02,2024-11-22 18:00:00
4812,Bottle,2022-09-28,NaN,7.82,10,2024-11-04 18:24:27,2024-11-11 18:00:00
4813,Grid Tales,2021-12-14,NaN,16.73,10,2024-11-04 18:23:54,2024-11-11 18:00:00
4814,My Wet Leto Comic-A Day of Green,2024-10-31,NaN,26.99,10,2024-10-31 15:10:02,2024-11-14 18:00:00


In [ ]:
df_dados.to_excel('dados_steamDB.xlsx', index=0)